# ***EXEMPLE DE TRAITEMENTS DES FICHIERS SOURCES***
> exemples pour les FIM, donnees individuelles mixtra, données individuelles viking, ...

In [120]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import pandas as pd
import geopandas as gp
import numpy as np
import altair as alt
from datetime import datetime
import locale

import Donnees_individuelles as di

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
locale.setlocale(locale.LC_TIME, 'fr_FR')

'fr_FR'

# 1. Données individuelles mixtra
ces données sont des fichiers excels, sans aucune référence interne au compteur. A chaque véhicule correspond une ligne avec les atributs descroiptifs

In [443]:
mixtra=di.Mixtra([r'P:\DAIT\TI\Donnees\1-ROUTIER\1 - Aquitaine\64 - Pyrénées-Atlantiques\2020\Biriatou\7_Juillet\Tubes\Individuelle\P4_TUBES_23_07_20\V0_02_B.XLS'],
                 [r'P:\DAIT\TI\Donnees\1-ROUTIER\1 - Aquitaine\64 - Pyrénées-Atlantiques\2020\Biriatou\7_Juillet\Tubes\Individuelle\P4_TUBES_23_07_20\V1_02_B.XLS'])

In [ ]:
#calculer la df qui va contenir toute les données horaires
dfValide=di.NettoyageTemps(mixtra.NettoyageDonnees())
dfHeureTypeSens=di.GroupeCompletude(dfValide)

In [548]:
#creer le dico qui va contenir tout les résultats et données intermeédiaires 
dicoHoraire,dicoJournalier=di.IndicsGraphsHoraires(dfHeureTypeSens,['vl','pl','2r'],['mja','mjo', 'samedi', 'dimanche'],['sens1','sens2','2sens'])

In [521]:
#exemple d'appel de résultats
dicoHoraire['mjo']['nbJour']
dicoHoraire['mja']['nbJour']
dicoHoraire['mjo']['2sens']['graph']
dicoHoraire['samedi']['sens1']['donnees']
dicoJournalier['mja']['type_veh']['graph']
dicoJournalier['mjo']['sens']['graph']

# 2. Données individuelles viking
ces données sont des fichiers excels, sans aucune référence interne au compteur. A chaque véhicule correspond une ligne avec les atributs descroiptifs

In [316]:
fichier1Sens=pd.read_csv(r'P:\DAIT\TI\Donnees\1-ROUTIER\1 - Aquitaine\64 - Pyrénées-Atlantiques\2020\Biriatou\7_Juillet\Radars\Brut\rad_S1.VIK',delimiter=' ',
                         skiprows=1, names=['sens', 'jour', 'heureMin','secCent', 'vts', 'ser', 'type_veh'],dtype={'heureMin':str,'secCent':str})

In [317]:
with open(r'P:\DAIT\TI\Donnees\1-ROUTIER\1 - Aquitaine\64 - Pyrénées-Atlantiques\2020\Biriatou\7_Juillet\Radars\Brut\rad_S1.VIK') as f :
    entete=[e.strip() for e in f.readlines()][0]

In [318]:
anneeDeb,moisDeb,jourDeb=(entete.split('.')[i].strip() for i in range(5,8))
date_debut=pd.to_datetime(f'20{annee}-{mois}-{jour}')

In [321]:
def creer_date(jourDeb,moisDeb,anneeDeb, jourMesure,heureMin,secCent) : 
    """
    creer la date d'acquisition. Attention : si comptage sur un mois entier ça ne marche pas
    """
    if jourMesure<int(jourDeb) : 
        moisDeb=str(int(moisDeb)+1)
    return pd.to_datetime(f'20{anneeDeb}-{moisDeb}-{jourMesure} {str(heureMin)[:2]}:{str(heureMin)[2:]}:{str(secCent)[:2]}.{str(secCent)[2:]}')
fichier1Sens['date_heure']=fichier1Sens.apply(lambda x : creer_date(jourDeb,moisDeb,anneeDeb, x['jour'],x['heureMin'],x['secCent']), axis=1)

In [322]:
fichier1Sens

,sens,jour,heureMin,secCent,vts,ser,type_veh,date_heure
0,1,8,1240,5764,V=035,S=0743,VL,2020-07-08 12:40:57.640
1,1,8,1244,4147,V=037,S=0700,VL,2020-07-08 12:44:41.470
2,1,8,1246,2915,V=036,S=0619,VL,2020-07-08 12:46:29.150
3,1,8,1247,1528,V=040,S=0768,VL,2020-07-08 12:47:15.280
4,1,8,1247,2937,V=039,S=0735,VL,2020-07-08 12:47:29.370
...,...,...,...,...,...,...,...,...
7681,1,23,1415,0307,V=032,S=0582,VL,2020-07-23 14:15:03.070
7682,1,23,1415,1605,V=039,S=0600,VL,2020-07-23 14:15:16.050
7683,1,23,1415,1846,V=037,S=0531,VL,2020-07-23 14:15:18.460
7684,1,23,1416,4365,V=036,S=0707,VL,2020-07-23 14:16:43.650


In [421]:
vik=di.Viking(r'P:\DAIT\TI\Donnees\1-ROUTIER\1 - Aquitaine\64 - Pyrénées-Atlantiques\2020\Biriatou\7_Juillet\Radars\Brut\rad_S1.VIK',
             r'P:\DAIT\TI\Donnees\1-ROUTIER\1 - Aquitaine\64 - Pyrénées-Atlantiques\2020\Biriatou\7_Juillet\Radars\Brut\rad_S2.VIK')

In [423]:
dfValide, nbJours, nbJoursEffectifs, nbJoursOuvrés=di.NettoyageTemps(vik.fichier2sens)

In [430]:
dfHeureTypeSens=di.GroupeCompletude(dfValide)

In [434]:
(dfMoyenneJourType,dfMoyenneJourSens,dfMoyenneHeureTypeSensMja,dfMoyenneHeureTypeSensMjo,
hpmJo,hpsJo,dfMoyenneHoraireNuit,dfMoyenneHoraireJour,tmja,tmjo)=di.Indicateurs(dfHeureTypeSens,nbJoursEffectifs,nbJoursOuvrés)
graphJourType,graphJourSens=di.GraphsMoyennesJours(dfMoyenneJourType,dfMoyenneJourSens)
(graphHeureType2SensMja,graphHeureType2SensMjo,graphHeureTypeSens1Mja,graphHeureTypeSens2Mja,
 graphHeureTypeSens1Mjo,graphHeureTypeSens2Mjo)=di.GraphsMoyenneshoraires(dfMoyenneHeureTypeSensMja,dfMoyenneHeureTypeSensMjo)

In [437]:
graphHeureType2SensMja

alt.Chart(...)

In [438]:
graphHeureType2SensMjo

alt.Chart(...)